[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ashrafalaghbari/GA-TCN-LSTM/blob/main/Data_pre-processing/data_cleaning.ipynb)


In [125]:
%autosave 60

Autosaving every 60 seconds


In [126]:
import numpy as np
import pandas as pd

In [127]:
df = pd.read_csv("F_14.csv",  parse_dates=["DATEPRD"], index_col="DATEPRD")

In [128]:
# remove average annulur pressure and pressure drop across the wellbore
df = df.drop(["AVG_ANNULUS_PRESS", "AVG_DP_TUBING"], axis=1)

In [129]:
#Correct the errors found in data points (24.31, 25) by changing the value to 24 hours.
# This is because the preceding and subsequent values of on-stream hours are 24, and the oil volume at these points is close to that at (24.31, 25) hours.
df["ON_STREAM_HRS"]  = np.where(df["ON_STREAM_HRS"] > 24 , 24, df["ON_STREAM_HRS"])

In [130]:
# the values for all variables are corrupted in the period of "2012-09-02" & "2012-09-28"
# on strem hours is set to zero becasue oil production is zero
df.loc["2012-09-02":"2012-09-28",'ON_STREAM_HRS'] = 0
# average reservoir presure in this period is corrupted
df.loc["2012-09-02":"2012-09-03",'AVG_DOWNHOLE_PRESSURE'] = np.nan
# # average reservori pressure is decreaseing and decreasing; therefore, is imputated with linear interpolation 
df.loc["2012-09-01":"2012-09-27",'AVG_DOWNHOLE_PRESSURE'].interpolate(inplace = True, limit_direction ='forward')
# # average reservoir temperature in this period is corrupted
df.loc["2012-09-02":"2012-09-03",'AVG_DOWNHOLE_TEMPERATURE'] = np.nan
# # average reservori pressure is decreaseing and decreasing; therefore, is imputated with linear interpolation 
df.loc["2012-09-01":"2012-09-27",'AVG_DOWNHOLE_TEMPERATURE'].interpolate(inplace = True,limit_direction ='forward')
# The average choke size should be set to zero when the well is not in production
df.loc[df['ON_STREAM_HRS'] == 0, 'AVG_CHOKE_SIZE_P'] = 0
df["2012-09-01":"2012-09-27"] = df["2012-09-01":"2012-09-27"].interpolate(limit_direction ='forward')
# use ffill to fill forward missing values
df.loc["2012-09-27":"2012-09-29"] = df.loc["2012-09-27":"2012-09-29"].fillna(method='ffill')

# the on stream injection volumne for F-5 on this day is zero, which means the injection rate should be zero too
df.loc["2012-09-28"]['F_5_BORE_WI_VOL'] = 0

mask = ((df["BORE_GAS_VOL"] == 0) | (df["BORE_WAT_VOL"] == 0) | (df["BORE_OIL_VOL"] == 0))
df["ON_STREAM_HRS"] = np.where(mask, 0, df["ON_STREAM_HRS"])


In [131]:
df.loc["2012-08-13":"2012-09-15"] = df.loc["2012-08-13":"2012-09-15"].bfill(axis=0)
df.loc["2012-08-17":"2012-09-19"] = df.loc["2012-08-17":"2012-09-19"].interpolate(limit_direction ='forward')

In [132]:
# replace the negative value in the water volume column with  positive one
df.loc["2012-08-13"].BORE_WAT_VOL = df.loc["2012-08-13"].BORE_WAT_VOL *-1

In [133]:
#  replace the corrupted values with NaN then impute the downhole pressure with linear interpolation and temperature with 
df.loc["2010-02-10":"2010-02-27","AVG_DOWNHOLE_PRESSURE"] =  df.loc["2010-02-10":"2010-02-27","AVG_DOWNHOLE_PRESSURE"].apply(lambda x: np.nan if x < 3300 else x)
df.loc["2010-02-09":"2010-02-27","AVG_DOWNHOLE_PRESSURE"] = df.loc["2010-02-09":"2010-02-27","AVG_DOWNHOLE_PRESSURE"].interpolate(
    method ='linear', limit_direction ='forward')
# downhole temperature
df.loc["2010-02-10":"2010-02-27","AVG_DOWNHOLE_TEMPERATURE"] = df["2010-02-10":"2010-02-27"]["AVG_DOWNHOLE_TEMPERATURE"].apply(lambda x:np.nan if x < 220  else x) 
df.loc["2010-02-10":"2010-02-27","AVG_DOWNHOLE_TEMPERATURE"] = df["2010-02-10":"2010-02-27"]["AVG_DOWNHOLE_TEMPERATURE"].apply(
    lambda x:np.nan if x < 220  else x).replace(
        np.nan,df.loc["2010-02-10":"2010-02-27","AVG_DOWNHOLE_TEMPERATURE"].mean() )

In [134]:
#  replace the corrupted values with NaN then impute the downhole pressure with linear interpolation and temperature with 
df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_PRESSURE"] =  df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_PRESSURE"].apply(lambda x: np.nan if x < 3000 else x)
df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_PRESSURE"] = df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_PRESSURE"].interpolate(
    method ='linear', limit_direction ='forward')
# downhole temperature
df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_TEMPERATURE"] = df["2013-09-26":"2013-09-30"]["AVG_DOWNHOLE_TEMPERATURE"].apply(lambda x:np.nan if x < 200  else x) 
df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_TEMPERATURE"] = df["2013-09-26":"2013-09-30"]["AVG_DOWNHOLE_TEMPERATURE"].apply(
    lambda x:np.nan if x < 200  else x).replace(
        np.nan,df.loc["2013-09-26":"2013-09-30","AVG_DOWNHOLE_TEMPERATURE"].mean() )


In [135]:
# fix downhole pressure/ temperature and pressure drop along the tubing
df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_PRESSURE"] =  df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_PRESSURE"].apply(lambda x: np.nan if x < 2000 else x)
df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_PRESSURE"] = df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_PRESSURE"].interpolate(
    method ='linear', limit_direction ='forward')
 # downhole temperature
df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_TEMPERATURE"] = df["2013-05-14":"2013-05-16"]["AVG_DOWNHOLE_TEMPERATURE"].apply(lambda x:np.nan if x < 200  else x) 
df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_TEMPERATURE"] = df["2013-05-14":"2013-05-16"]["AVG_DOWNHOLE_TEMPERATURE"].apply(
    lambda x:np.nan if x < 200  else x).replace(
        np.nan,df.loc["2013-05-14":"2013-05-16","AVG_DOWNHOLE_TEMPERATURE"].mean() )


In [136]:
df.loc["2010-01-19":"2010-01-21","AVG_DOWNHOLE_TEMPERATURE"] = df["2010-01-19":"2010-01-21"].AVG_DOWNHOLE_TEMPERATURE.apply(
    lambda x:np.nan if x < 200 else x).interpolate( method ='linear', limit_direction ='forward')
df.loc["2010-01-19":"2010-01-21","AVG_DOWNHOLE_PRESSURE"] = df["2010-01-19":"2010-01-21"].AVG_DOWNHOLE_PRESSURE.apply(
lambda x:np.nan if x < 3000 else x).interpolate( method ='linear', limit_direction ='forward')

In [137]:
df.loc["2013-07-07":"2013-07-09","AVG_DOWNHOLE_TEMPERATURE"] = df["2013-07-07":"2013-07-09"].AVG_DOWNHOLE_TEMPERATURE.apply(
    lambda x:np.nan if x < 200 else x).interpolate( method ='linear', limit_direction ='forward')
df.loc["2013-07-07":"2013-07-09","AVG_DOWNHOLE_PRESSURE"] = df["2013-07-07":"2013-07-09"].AVG_DOWNHOLE_PRESSURE.apply(
lambda x:np.nan if x < 3100 else x).interpolate( method ='linear', limit_direction ='forward')

In [138]:


df.loc["2014-06-25":"2014-06-28","AVG_DOWNHOLE_TEMPERATURE"] = df["2014-06-25":"2014-06-28"].AVG_DOWNHOLE_TEMPERATURE.apply(
    lambda x:np.nan if x < 200 else x).interpolate( method ='linear', limit_direction ='forward')
df.loc["2014-06-25":"2014-06-28","AVG_DOWNHOLE_PRESSURE"] = df["2014-06-25":"2014-06-28"].AVG_DOWNHOLE_PRESSURE.apply(
lambda x:np.nan if x < 3000 else x).interpolate( method ='linear', limit_direction ='forward')

In [139]:
df.loc["2014-08-27":"2014-08-30","AVG_DOWNHOLE_TEMPERATURE"] = df["2014-08-27":"2014-08-30"].AVG_DOWNHOLE_TEMPERATURE.apply(
    lambda x:np.nan if x < 200 else x).interpolate( method ='linear', limit_direction ='forward')
df.loc["2014-08-27":"2014-08-30","AVG_DOWNHOLE_PRESSURE"] = df["2014-08-27":"2014-08-30"].AVG_DOWNHOLE_PRESSURE.apply(
lambda x:np.nan if x < 3000 else x).interpolate( method ='linear', limit_direction ='forward')

In [140]:
df.loc[df.F_4_ON_STREAM_HRS == 0, "F_4_BORE_WI_VOL"] = 0
df.loc[df.F_5_ON_STREAM_HRS == 0, "F_5_BORE_WI_VOL"] = 0
df.loc[df.F_4_BORE_WI_VOL == 0, "F_4_ON_STREAM_HRS"] = 0
df.loc[df.F_5_BORE_WI_VOL == 0, "F_5_ON_STREAM_HRS"] = 0

In [141]:
df.loc[df['F_4_ON_STREAM_HRS'] > 24, 'F_4_ON_STREAM_HRS'] = 24
df.loc[df['F_5_ON_STREAM_HRS'] > 24, 'F_5_ON_STREAM_HRS'] = 24

In [142]:
df.interpolate(inplace=True, method='linear')

In [143]:
df.to_csv("cleaned_F_14.csv")